# Цель:
Очистить список email-адресов от ошибок для подготовки к email-рассылке. 
# Решение:
1. Получил данные из Excel
2. Провел первичный анализ датасета
3. Сформулировал и проверил гипотезы о возможных ошибках
4. Выполнил очистку адресов:
   - Удалил дубликаты email
   - Удалил лишние пробелы (в начале, конце или середине адреса)
   - Привел все буквы нижнему регистру
   - Заменил запятые на точки в доменной части
   - Поместил в отдельный список тестовые адреса (содержат комбинацию test и доменное имя магазина)
5. Выгрузил очищенный датасет в csv формат

Все персональные данные анонимизированы. Использован синтетический датасет, воспроизводящий паттерны ошибок из оригинальных данных.

## Загрузка библиотек и файла

In [332]:
import pandas as pd

df = pd.read_excel('emails_dataset.xlsx')
df = df.rename(columns={'Электронные адреса клиентов:': 'email'})

## Первичный анализ 

In [334]:
df

,email
0,7y5bn@mail.ru
1,g1tg-95_vqc@yandex.ru
2,a-k-l88r@yandex.ru
3,9awzw7@mail.ru
4,tgzd_9-n01@yandex.ru
...,...
1003,0r6tqaxwx@24shop.by
1004,test81@24shop.by
1005,1g2ir3tuvwg@gmail.com
1006,m_ijn7xjt@mail.ru


In [336]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   1007 non-null   object
dtypes: object(1)
memory usage: 8.0+ KB


In [338]:
# Анализ длины email-адресов
df['email'].str.len().describe()

count    1007.000000
mean       18.150943
std         2.692131
min         1.000000
25%        16.000000
50%        18.000000
75%        20.000000
max        33.000000
Name: email, dtype: float64

## Выявление ошибок

In [341]:
invalid_emails = df[~df['email'].str.contains('@', na=False)]
print(f"Email без @: {len(invalid_emails)}")

spaced_emails = df[df['email'].str.contains(' ', na=False)]
print(f"Email с пробелами: {len(spaced_emails)}")

comma_emails = df[df['email'].str.contains(',', na=False)]
print(f"Email с запятыми: {len(comma_emails)}")

test_emails = df[df['email'].str.contains('test', case=False, na=False) 
                & df['email'].str.contains('@24shop.by', case=False, na=False)]
print(f"Тестовые email: {len(test_emails)}")

print(f"Дубликаты: {df.duplicated().sum()}")

Email без @: 7
Email с пробелами: 4
Email с запятыми: 11
Тестовые email: 53
Дубликаты: 13


## Решение

In [344]:
emails = df['email'].tolist() # датасет преобразую в список

In [346]:
emails_clean = set() 
emails_test = set()

for email in emails:
    if pd.isna(email) or not isinstance(email, str): # пропускаю NaN и нестроковые значения
        continue
    if '@' not in email: #проверяю наличие @
        continue    
    normalized_email = email.lower().replace(" ", "").replace(",", ".") # нижний регистр, убираю пробелы, запятую меняю на точку
    if 'test' in normalized_email and '@24shop.by' in normalized_email:
        emails_test.add(normalized_email)    
    else:
        emails_clean.add(normalized_email)

# Для визуального контроля
print("-----\nТестовые:", emails_test)
print("-----\nЧистые:", emails_clean)

-----
Тестовые: {'test87@24shop.by', 'test1@24shop.by', 'test25@24shop.by', 'test35@24shop.by', 'test91@24shop.by', 'test9@24shop.by', 'test52@24shop.by', 'test56@24shop.by', 'test94@24shop.by', 'test19@24shop.by', 'test54@24shop.by', 'test97@24shop.by', 'test58@24shop.by', 'test46@24shop.by', 'test71@24shop.by', 'test55@24shop.by', 'test83@24shop.by', 'test49@24shop.by', 'test14@24shop.by', 'test93@24shop.by', 'test21@24shop.by', 'test48@24shop.by', 'test61@24shop.by', 'test33@24shop.by', 'test42@24shop.by', 'test81@24shop.by', 'test27@24shop.by', 'test60@24shop.by', 'test24@24shop.by', 'test57@24shop.by', 'test44@24shop.by', 'test29@24shop.by', 'test8@24shop.by', 'test51@24shop.by', 'test65@24shop.by', 'test32@24shop.by', 'test10@24shop.by', 'test92@24shop.by', 'test22@24shop.by', 'test100@24shop.by', 'test68@24shop.by'}
-----
Чистые: {'sccsn48w50-w@24shop.by', 'hl-a7z3ksh@24shop.by', 'q9w5v@mail.ru', 'mbthf0clvj3u@24shop.by', 'g8g-lo@24shop.by', 'zi-sdyv@yandex.ru', 'wq0p7wfpl_@mail

## Выгрузка в CSV

In [349]:
pd.DataFrame(list(emails_clean)).to_csv('emails_clean.csv', index=False, header=False) #в csv, без заголовка

print(f"Получено из Excel {len(df)} строк")
print(f"Выгружено в CSV {len(emails_clean)} чистых email")
print(f"Отбраковано {len(emails_test)} тестовых email")

Получено из Excel 1008 строк
Выгружено в CSV 947 чистых email
Отбраковано 41 тестовых email
